In [218]:
import requests
import pandas as pd
import plotly.express as px
import datetime
import numpy as np
import plotly.graph_objects as go

In [219]:
def download_esios_id(id,fecha_ini,fecha_fin,agrupacion):
                       token = '496b263791ef0dcaf80b803b47b332a13b01f2c2352e018b624c7a36a0eaffc0'
                       cab = dict()
                       cab ['x-api-key']= token
                       url_id = 'https://api.esios.ree.es/indicators'
                       url=f'{url_id}/{id}?geo_ids[]=3&start_date={fecha_ini}T00:00:00&end_date={fecha_fin}T23:59:59&time_trunc={agrupacion}'
                       print(url)
                       response = requests.get(url, headers=cab).json()
                       

                       return response
                       

In [220]:
id='600'
fecha_ini='2024-01-01'
fecha_fin='2024-12-31'
agrupacion='hour'

In [221]:
datos_origen =download_esios_id(id,fecha_ini,fecha_fin,agrupacion)


https://api.esios.ree.es/indicators/600?geo_ids[]=3&start_date=2024-01-01T00:00:00&end_date=2024-12-31T23:59:59&time_trunc=hour


In [222]:
datos=pd.DataFrame(datos_origen['indicator']['values'])
datos

,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,63.33,2024-01-01T00:00:00.000+01:00,2023-12-31T23:00:00Z,2023-12-31T23:00:00.000Z,3,España
1,50.09,2024-01-01T01:00:00.000+01:00,2024-01-01T00:00:00Z,2024-01-01T00:00:00.000Z,3,España
2,47.50,2024-01-01T02:00:00.000+01:00,2024-01-01T01:00:00Z,2024-01-01T01:00:00.000Z,3,España
3,43.50,2024-01-01T03:00:00.000+01:00,2024-01-01T02:00:00Z,2024-01-01T02:00:00.000Z,3,España
4,42.50,2024-01-01T04:00:00.000+01:00,2024-01-01T03:00:00Z,2024-01-01T03:00:00.000Z,3,España
...,...,...,...,...,...,...
4530,35.94,2024-07-07T19:00:00.000+02:00,2024-07-07T17:00:00Z,2024-07-07T17:00:00.000Z,3,España
4531,59.57,2024-07-07T20:00:00.000+02:00,2024-07-07T18:00:00Z,2024-07-07T18:00:00.000Z,3,España
4532,100.73,2024-07-07T21:00:00.000+02:00,2024-07-07T19:00:00Z,2024-07-07T19:00:00.000Z,3,España
4533,104.58,2024-07-07T22:00:00.000+02:00,2024-07-07T20:00:00Z,2024-07-07T20:00:00.000Z,3,España


In [223]:
datos = (datos
         .assign(datetime=lambda vh_: pd #formateamos campo fecha, desde un str con diferencia horaria a un naive
                      .to_datetime(vh_['datetime'],utc=True)  # con la fecha local
                      .dt
                      .tz_convert('Europe/Madrid')
                      .dt
                      .tz_localize(None)
                ) 
             #.drop(['datetime','datetime_utc','tz_time','geo_id','geo_name'],
             #      axis=1) #eliminamos campos
             
             .loc[:,['datetime','value']]
             )

datos

,datetime,value
0,2024-01-01 00:00:00,63.33
1,2024-01-01 01:00:00,50.09
2,2024-01-01 02:00:00,47.50
3,2024-01-01 03:00:00,43.50
4,2024-01-01 04:00:00,42.50
...,...,...
4530,2024-07-07 19:00:00,35.94
4531,2024-07-07 20:00:00,59.57
4532,2024-07-07 21:00:00,100.73
4533,2024-07-07 22:00:00,104.58


In [224]:
meses = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

### Esta es la tabla de valores horarios tratada

In [225]:
datos['fecha']=datos['datetime'].dt.strftime('%d/%m/%Y')
datos['fecha']=pd.to_datetime(datos['fecha'],format='%d/%m/%Y')
datos['hora']=datos['datetime'].dt.hour
datos['dia']=datos['datetime'].dt.day
datos['mes']=datos['datetime'].dt.month
datos['año']=datos['datetime'].dt.year
#datos['mes_nombre']=datos['datetime'].dt.month_name()
#datos['mes_nombre'] = datos['mes_nombre'].map(meses)

datos.set_index('datetime', inplace=True)
datos

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024
...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024


In [226]:
valor_minimo_horario=datos['value'].min()
valor_maximo_diario=datos['value'].max()
valor_minimo_horario,valor_maximo_diario

(-2.0, 181.26)

### Copia para escala de colores

In [227]:
datos_horarios=datos
datos_horarios


,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024
...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024


### Agrupación por días

In [228]:

datos_dia=datos.resample('D').mean()
datos_dia['value']=datos_dia['value'].round(2)
datos_dia=datos_dia.drop(columns=['hora'])
datos_dia[['dia','mes','año']]=datos_dia[['dia','mes','año']].astype(int)
datos_dia

,value,fecha,dia,mes,año
datetime,,,,,
2024-01-01,36.24,2024-01-01,1,1,2024
2024-01-02,40.91,2024-01-02,2,1,2024
2024-01-03,55.08,2024-01-03,3,1,2024
2024-01-04,91.36,2024-01-04,4,1,2024
2024-01-05,68.62,2024-01-05,5,1,2024
...,...,...,...,...,...
2024-07-03,61.71,2024-07-03,3,7,2024
2024-07-04,51.85,2024-07-04,4,7,2024
2024-07-05,59.88,2024-07-05,5,7,2024


In [229]:
valor_minimo_diario=datos_dia['value'].min()
valor_maximo_diario=datos_dia['value'].max()
valor_minimo_diario,valor_maximo_diario

(0.44, 113.83)

### Agrupación por meses

In [230]:
datos_mes=datos.resample('M').mean()
datos_mes['value']=datos_mes['value'].round(2)
datos_mes=datos_mes.drop(columns=['fecha','hora', 'dia'])
datos_mes[['mes','año']]=datos_mes[['mes','año']].astype(int)
datos_mes

,value,mes,año
datetime,,,
2024-01-31,74.10,1,2024
2024-02-29,40.00,2,2024
2024-03-31,20.31,3,2024
2024-04-30,13.67,4,2024
2024-05-31,30.40,5,2024
2024-06-30,56.08,6,2024
2024-07-31,48.90,7,2024


def graf_año():
    graf_año=px.line(datos_dia, x='fecha', y='value')
    return graf_año

### Definimos la escala y sus límites

In [231]:
datos_limites = {
    'rango': [-10,20,40,60,80,100,10000],
    'valor_asignado': ['muy bajo', 'bajo','medio','alto','muy alto','chungo','xtrem'],
}

In [232]:
df_limites=pd.DataFrame(datos_limites)
df_limites

,rango,valor_asignado
0,-10,muy bajo
1,20,bajo
2,40,medio
3,60,alto
4,80,muy alto
5,100,chungo
6,10000,xtrem


In [233]:
#etiquetas = df_limites['valor_asignado'][:-1]
etiquetas = df_limites['valor_asignado'][:-1]
etiquetas

0    muy bajo
1        bajo
2       medio
3        alto
4    muy alto
5      chungo
Name: valor_asignado, dtype: object

### Añadimos la columna escala a todas las tablas

In [234]:
datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
#datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=etiquetas)
#datos_horarios['escala']=pd.cut(datos_horarios['value'],bins=df_limites['rango'],labels=df_limites['valor_asignado'],right=False)
datos_horarios

,value,fecha,hora,dia,mes,año,escala
datetime,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio
...,...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024,bajo
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024,medio
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024,chungo


In [235]:
lista_escala=datos_horarios['escala'].unique()
lista_escala

['alto', 'medio', 'bajo', 'muy bajo', 'muy alto', 'chungo']
Categories (6, object): ['muy bajo' < 'bajo' < 'medio' < 'alto' < 'muy alto' < 'chungo']

In [236]:
datos_dia['escala']=pd.cut(datos_dia['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
datos_dia

,value,fecha,dia,mes,año,escala
datetime,,,,,,
2024-01-01,36.24,2024-01-01,1,1,2024,bajo
2024-01-02,40.91,2024-01-02,2,1,2024,medio
2024-01-03,55.08,2024-01-03,3,1,2024,medio
2024-01-04,91.36,2024-01-04,4,1,2024,muy alto
2024-01-05,68.62,2024-01-05,5,1,2024,alto
...,...,...,...,...,...,...
2024-07-03,61.71,2024-07-03,3,7,2024,alto
2024-07-04,51.85,2024-07-04,4,7,2024,medio
2024-07-05,59.88,2024-07-05,5,7,2024,medio


In [237]:
datos_mes['escala']=pd.cut(datos_mes['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
datos_mes

,value,mes,año,escala
datetime,,,,
2024-01-31,74.10,1,2024,alto
2024-02-29,40.00,2,2024,medio
2024-03-31,20.31,3,2024,bajo
2024-04-30,13.67,4,2024,muy bajo
2024-05-31,30.40,5,2024,bajo
2024-06-30,56.08,6,2024,medio
2024-07-31,48.90,7,2024,medio


### Definimos los colores según la escala

In [238]:
colores = {
    'muy bajo': 'lightgreen',
    'bajo': 'green',
    'medio': 'blue',
    'alto': 'orange',
    'muy alto': 'red',
    'chungo': 'purple',
    'xtrem':'black'
}

In [239]:
datos_horarios['color']=datos_horarios['escala'].map(colores)
datos_horarios

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto,orange
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio,blue
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio,blue
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio,blue
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio,blue
...,...,...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024,bajo,green
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024,medio,blue
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024,chungo,purple


In [240]:
datos_dia['color']=datos_dia['escala'].map(colores)
datos_dia

,value,fecha,dia,mes,año,escala,color
datetime,,,,,,,
2024-01-01,36.24,2024-01-01,1,1,2024,bajo,green
2024-01-02,40.91,2024-01-02,2,1,2024,medio,blue
2024-01-03,55.08,2024-01-03,3,1,2024,medio,blue
2024-01-04,91.36,2024-01-04,4,1,2024,muy alto,red
2024-01-05,68.62,2024-01-05,5,1,2024,alto,orange
...,...,...,...,...,...,...,...
2024-07-03,61.71,2024-07-03,3,7,2024,alto,orange
2024-07-04,51.85,2024-07-04,4,7,2024,medio,blue
2024-07-05,59.88,2024-07-05,5,7,2024,medio,blue


### Obtenemos la escala ordenada al revés para el gráfico horario

In [241]:
valor_asignado_a_rango = {row['valor_asignado']: row['rango'] for _, row in df_limites.iterrows()}

In [242]:
escala_horaria=['alto', 'medio', 'bajo', 'muy bajo', 'muy alto', 'chungo']
escala_horaria

['alto', 'medio', 'bajo', 'muy bajo', 'muy alto', 'chungo']

In [243]:
escala_ordenada_hora = sorted(escala_horaria, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_hora

['chungo', 'muy alto', 'alto', 'medio', 'bajo', 'muy bajo']

In [244]:
datos_horarios['color']=datos_horarios['escala'].map(colores)
datos_horarios

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto,orange
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio,blue
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio,blue
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio,blue
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio,blue
...,...,...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024,bajo,green
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024,medio,blue
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024,chungo,purple


In [245]:
datos_horarios['escala']=pd.Categorical(datos_horarios['escala'],categories=escala_ordenada_hora, ordered=True)
datos_horarios

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto,orange
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio,blue
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio,blue
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio,blue
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio,blue
...,...,...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024,bajo,green
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024,medio,blue
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024,chungo,purple


### Obtenemos la escala ordenada al reves para el gráfico diario

In [246]:
escala_dia=datos_dia['escala'].unique()
escala_dia

['bajo', 'medio', 'muy alto', 'alto', 'chungo', 'muy bajo']
Categories (6, object): ['muy bajo' < 'bajo' < 'medio' < 'alto' < 'muy alto' < 'chungo']

In [247]:
escala_ordenada_dia = sorted(escala_dia, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_dia

['chungo', 'muy alto', 'alto', 'medio', 'bajo', 'muy bajo']

In [248]:
datos_mes['color']=datos_mes['escala'].map(colores)
datos_mes

,value,mes,año,escala,color
datetime,,,,,
2024-01-31,74.10,1,2024,alto,orange
2024-02-29,40.00,2,2024,medio,blue
2024-03-31,20.31,3,2024,bajo,green
2024-04-30,13.67,4,2024,muy bajo,lightgreen
2024-05-31,30.40,5,2024,bajo,green
2024-06-30,56.08,6,2024,medio,blue
2024-07-31,48.90,7,2024,medio,blue


### Obtenemos la escala ordenada al reves para el gráfico mensual

In [249]:
escala_mes= datos_mes['escala'].unique()
escala_mes

['alto', 'medio', 'bajo', 'muy bajo']
Categories (6, object): ['muy bajo' < 'bajo' < 'medio' < 'alto' < 'muy alto' < 'chungo']

In [250]:
escala_ordenada_mes = sorted(escala_mes, key=lambda x: valor_asignado_a_rango[x], reverse=True)
escala_ordenada_mes

['alto', 'medio', 'bajo', 'muy bajo']

### Esta tabla se usa para el gráfico grande de barras diario

In [251]:

datos_dia['escala']=pd.Categorical(datos_dia['escala'],categories=escala_ordenada_dia, ordered=True)
datos_dia

,value,fecha,dia,mes,año,escala,color
datetime,,,,,,,
2024-01-01,36.24,2024-01-01,1,1,2024,bajo,green
2024-01-02,40.91,2024-01-02,2,1,2024,medio,blue
2024-01-03,55.08,2024-01-03,3,1,2024,medio,blue
2024-01-04,91.36,2024-01-04,4,1,2024,muy alto,red
2024-01-05,68.62,2024-01-05,5,1,2024,alto,orange
...,...,...,...,...,...,...,...
2024-07-03,61.71,2024-07-03,3,7,2024,alto,orange
2024-07-04,51.85,2024-07-04,4,7,2024,medio,blue
2024-07-05,59.88,2024-07-05,5,7,2024,medio,blue


### Estos datos se usan para el grafico de barras mensual

In [252]:
datos_mes['escala']=pd.Categorical(datos_mes['escala'],categories=escala_ordenada_mes, ordered=True)
datos_mes

,value,mes,año,escala,color
datetime,,,,,
2024-01-31,74.10,1,2024,alto,orange
2024-02-29,40.00,2,2024,medio,blue
2024-03-31,20.31,3,2024,bajo,green
2024-04-30,13.67,4,2024,muy bajo,lightgreen
2024-05-31,30.40,5,2024,bajo,green
2024-06-30,56.08,6,2024,medio,blue
2024-07-31,48.90,7,2024,medio,blue


### Estos datos se usan para el quesito

In [253]:
datos_dia_queso=datos_dia.groupby(['escala'])['escala'].count()
datos_dia_queso=datos_dia_queso.reset_index(name='num_dias')
datos_dia_queso


,escala,num_dias
0,chungo,3
1,muy alto,19
2,alto,28
3,medio,43
4,bajo,34
5,muy bajo,62


### Gráfico de barras principal

In [254]:
def graf_ecv_anual():
    graf_ecv_anual=px.bar(datos_dia, x='fecha', y='value', 
        color='escala',
        color_discrete_map=colores,
        category_orders={'escala':escala_ordenada_dia},
        labels={'value':'precio medio diario €/MWh', 'escala':'escala_cv'},
        title="Precios medios del mercado diario OMIE. Año 2024")
    graf_ecv_anual.update_xaxes(
        showgrid=True
    )
    graf_ecv_anual.update_traces(
        marker_line_width=0
    )

    return graf_ecv_anual

In [255]:
graf_ecv_anual()

In [256]:
def graf_ecv_mensual():
    graf_ecv_mensual=px.bar(datos_mes, x='mes', y='value',
        color='escala',
        color_discrete_map=colores,
        category_orders={'escala':escala_ordenada_mes},
        labels={'value':'precio medio mensual €/MWh', 'escala':'escala_cv'},
        title="Precios medios mensuales. Año 2024"
        )
    #graf_ecv_mensual.update_xaxes(
    #    showgrid=True
    
    #graf_ecv_mensual.update_traces(
    #   marker_line_width=0
    #)

    return graf_ecv_mensual

In [257]:
graf_ecv_mensual()

### Gráfico de queso

In [258]:
def graf_ecv_anual_queso():
    graf_ecv_anual_queso=px.pie(datos_dia_queso, values='num_dias', names='escala',
        color='escala',
        color_discrete_map=colores,
        #marker=dict(colors=colores),
        category_orders={'escala':escala_ordenada_dia},
        labels={'num_dias':'num_dias', 'escala':'escala_cv'},
        title="% y número de días según la Escala CV. Año 2024",
        width=500
        )
    
    return graf_ecv_anual_queso

In [259]:
graf_ecv_anual_queso()

### Gráfica horaria anual

In [260]:
pt_curva_horaria=datos.pivot_table(
    values='value',
    index='hora'
)
pt_curva_horaria=pt_curva_horaria['value'].round(2)
pt_curva_horaria=pt_curva_horaria.reset_index()


In [261]:
pt_curva_horaria

,hora,value
0,0,46.49
1,1,41.84
2,2,39.45
3,3,38.19
4,4,37.38
5,5,39.16
6,6,44.41
7,7,54.29
8,8,51.07
9,9,39.15


In [262]:
datos_horarios

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto,orange
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio,blue
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio,blue
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio,blue
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio,blue
...,...,...,...,...,...,...,...,...
2024-07-07 19:00:00,35.94,2024-07-07,19,7,7,2024,bajo,green
2024-07-07 20:00:00,59.57,2024-07-07,20,7,7,2024,medio,blue
2024-07-07 21:00:00,100.73,2024-07-07,21,7,7,2024,chungo,purple


In [271]:
def graf_horaria():
    graf_horaria=px.scatter(datos_horarios, x='hora',y='value',
        title='Perfil horario. Año 2024',                            
        animation_frame='fecha',
        width=800,
        labels={'value':'€/MWh'}
        #category_orders={'escala':escala_ordenada_hora},
        #color_discrete_map=colores
        #marker_size=10
    )
                           
    graf_horaria_linea=go.Scatter(
        x=pt_curva_horaria['hora'],
        y=pt_curva_horaria['value'],
        #color=["blue"],
        name='Media Anual',
        mode='lines',
        
    )
     
    graf_horaria.add_trace(graf_horaria_linea)
        
    graf_horaria.update_layout(
        yaxis=dict(range=[datos_horarios['value'].min(), datos_horarios['value'].max()])
    )
    
    return graf_horaria
graf_horaria()

In [265]:
mes=1
filtro_mes=datos['mes']==mes
datos_filtrados=datos[filtro_mes]
filtro_mes

datetime
2024-01-01 00:00:00     True
2024-01-01 01:00:00     True
2024-01-01 02:00:00     True
2024-01-01 03:00:00     True
2024-01-01 04:00:00     True
                       ...  
2024-07-07 19:00:00    False
2024-07-07 20:00:00    False
2024-07-07 21:00:00    False
2024-07-07 22:00:00    False
2024-07-07 23:00:00    False
Name: mes, Length: 4535, dtype: bool

In [266]:
datos_filtrados

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024,alto,orange
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024,medio,blue
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024,medio,blue
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024,medio,blue
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024,medio,blue
...,...,...,...,...,...,...,...,...
2024-01-31 19:00:00,119.47,2024-01-31,19,31,1,2024,chungo,purple
2024-01-31 20:00:00,168.35,2024-01-31,20,31,1,2024,chungo,purple
2024-01-31 21:00:00,94.54,2024-01-31,21,31,1,2024,muy alto,red


In [267]:
#datos_filtrados.set_index('datetime', inplace=True)
#datos_filtrados

In [268]:
datos_filtrados.dtypes

value            float64
fecha     datetime64[ns]
hora               int32
dia                int32
mes                int32
año                int32
escala          category
color           category
dtype: object

datos_filtrados_dia=datos_filtrados.resample('D').mean()
datos_filtrados_dia

def graf_dia():
    graf_dia=px.line(datos_filtrados_dia,x='fecha',y='value')
    
    return graf_dia

graf_dia()